### Syllabification Notebook
- This notebook takes WAV datasets generated by `1.0-segment-song-from-wavs` and segments the WAVs into spectrograms of syllables 
  - WAVs are expected to be in this format: `2017-04-16_17-27-44-760000.wav`
- The notebook outputs and HDF5 file which contains metadata about who the individual is, when the syllable was sung, how long the syllable is, which file the syllable comes from

### Import Packages

In [1]:
%load_ext autoreload
%autoreload 2

In [69]:
import numpy as np
from datetime import datetime
import pandas as pd
import copy
from glob import glob

from avgn.utils.paths import DATA_DIR, ensure_dir
from avgn.utils.general import save_dict_pickle
from avgn.signalprocessing.spectrogramming import _build_mel_basis

### Parameters for segmenting syllables

In [50]:
# the size of the syllables (pixels*pixels)
syll_size = 128

# parameters for filtering
filtering_params = {
    # filtering
    "highcut": 15000,
    "lowcut": 500,
}

spectrogramming_params = {
    # spectrograms
    "mel_filter": True,  # should a mel filter be used?
    "num_mels": syll_size,  # how many channels to use in the mel-spectrogram
    "num_freq": 512,  # how many channels to use in a spectrogram
    "num_freq_final": syll_size,  # how many channels to use in the resized spectrogram
    "sample_rate": 44100,  # what rate are your WAVs sampled at?
    "preemphasis": 0.97,
    "min_silence_for_spec": 0.5,  # minimum length of silence for a spectrogram to be considered a good spectrogram
    "max_vocal_for_spec": 5.0,  # the longest a single vocalization (protosyllable) is allowed to be
    "frame_shift_ms": 0.5,  # step size for fft
    "frame_length_ms": 6,  # frame length for fft
    "min_level_dB": -80,  # minimum threshold db for computing spe
    "min_level_dB_floor": -20,  # (db)
    "spec_thresh_delta_dB": 5,  # (db) what
    "ref_level_dB": 20,  # reference db for computing spec
    "sample_rate": 44100,  # sample rate of your data
    "fmin_mel": 300,  # low frequency cutoff for mel filter
    "fmax_mel": None,  # high frequency cutoff for mel filter
}

envelope_params = {
    # Vocal Envelope
    "smoothing": "gaussian",  # 'none',
    "envelope_signal": "spectrogram",  # spectrogram or waveform, what to get the vocal envelope from
    "gauss_sigma_s": 0.0001,
    "FOI_min": 4,  # minimum frequency of interest for vocal envelope (in terms of mel)
    "FOI_max": 24,  # maximum frequency of interest for vocal envelope (in terms of mel)
}

bout_threshold_params = {
    # Silence Thresholding
    "silence_threshold": 0.001,  # normalized threshold for silence
    "min_len": 5.0,  # minimum length for a vocalization (fft frames)
    "power_thresh": 0.3,  # Threshold for which a syllable is considered to be quiet weak and is probably noise
}

syllabification_params = {
    # Syllabification
    "min_syll_len_s": 0.03,  # minimum length for a syllable
    "segmentation_rate": 0.0,  # 0.125, # rate at which to dynamically raise the segmentation threshold (ensure short syllables)
    "threshold_max": 0.25,
    "min_num_sylls": 20,  # min number of syllables to be considered a bout
    "slow_threshold": 0.0,  # 0.02, # second slower threshold
    "max_size_syll": syll_size,  # the size of the syllable
    "resize_samp_fr": int(
        syll_size * 5.0
    ),  # (frames/s) the framerate of the syllable (in compressed spectrogram time components)
    # Sencond pass syllabification
    "second_pass_threshold_repeats": 50,  # the number of times to repeat the second pass threshold
    "ebr_min": 0.05,  # expected syllabic rate (/s) low
    "ebr_max": 0.2,  # expected syllabic rate (/s) high
    "max_thresh": 0.02,  # maximum pct of syllabic envelope to threshold at in second pass
    "thresh_delta": 0.005,  # delta change in threshold to match second pass syllabification
    "slow_threshold": 0.005,  # starting threshold for second pass syllabification
}

spectrogram_inversion_params = {
    # spectrogram inversion
    "max_iters": 200,
    "griffin_lim_iters": 60,
    "power": 1.5,
    # Thresholding out noise
    "mel_noise_filt": 0.15,  # thresholds out low power noise in the spectrum - higher numbers will diminish inversion quality
}

hparams = {
    'species': 'BF'
}

for d in [
    filtering_params,
    spectrogramming_params,
    envelope_params,
    bout_threshold_params,
    syllabification_params,
    spectrogram_inversion_params,
]:
    for k, v in d.items():
        hparams[k] = v

In [51]:
# this is used to identify this training instance
now_string = datetime.now().strftime(
    "%Y-%m-%d_%H-%M-%S"
)  
# save the dictionary so that we can reload it for recovering waveforms
dict_save = DATA_DIR / ("parameter_dictionaries/" + now_string + "_dict.pickle")
ensure_dir(dict_save)
save_dict_pickle(hparams, dict_save)
print(dict_save)

/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/parameter_dictionaries/2019-04-25_14-35-50_dict.pickle


## Segment bouts

#### Run through in parallel

In [70]:
# build a basis function if you are using a mel spectrogram
_mel_basis = _build_mel_basis(hparams) 

In [71]:
# find the data bird folders
dataset_location =   DATA_DIR / 'bf_wav/'
indv_folders = list(dataset_location.glob('*'))
indv_folders

[PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird4'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird3'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird9'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird7'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird0'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird8'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird2'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird5'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird10'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/AVGN_419/AVGN/data/bf_wav/Bird6')]

In [72]:
# where data will be written to
data_output_location = DATA_DIR / 'bf_song_syllables/'

In [73]:
# skip creating datasets that already exist
skip_existing = False 

# run through WAVs in parallel
parallel = True 

# how many threads to use if parallel is true
n_jobs = 10 

# how verbose to make the output of the parallelization (higher = more, 0 = none, >50 output is sent to std.out)
verbosity = 5 

# visualize the output of the algorithm for optimizing parameters
visualize = False 

# whether to save the dataset
save_dataset=True 

In [74]:
# visualization
nex = 10 # how many example wavs to plot

# make sure not to save datasets of the example plots
if visualize==True: 
    save_dataset=False

In [75]:
# the fields saved in the HDF5 file 
key_list = (
    'wav_file_names', # Wav file (bout_raw) that the syllable came from
    'syllable_spectrograms', # spectrogram of syllable
    'syll_start_datetimes', # time that this syllable occured
    'syll_start_rel_wav', # time relative to bout file that this 
    'syllable_lengths' # length of the syllable
   ) 

In [76]:
from tqdm.autonotebook import tqdm
import os

In [77]:
# prepare the data folder
species_dir = data_output_location / hparams["species"]
ensure_dir(species_dir)

# loop through and make individual datasets
for indv_folder in tqdm(indv_folders):

    # get the birds name
    indv_name = indv_folder.name

    # check if the file already exists
    individual_dataset_loc = species_dir / (indv_name + "_" + str(syll_size) + ".hdf5")
    if os.path.exists(individual_dataset_loc) and skip_existing:
        print("%s already complete, skipping" % (indv_name))
        continue

    # initialize lists of bird information
    indv_data = {key: [] for key in key_list}
    wav_list = list((indv_folder / "wavs").glob("*.wav"))
    print(indv_name, len(wav_list))

    # if visualizing, make sure only to show a few elements
    if visualize == True:
        wav_list = wav_list[:nex]

    # loop through each bout wav extracting syllable data
    if parallel:
        with Parallel() as parallel:
            syllable_data = delayed(
                sylls_from_bout(wav_file, _mel_basis, hparams, visualize=visualize)
                for wav_file in tqdm(wav_list)
            )
    else:
        syllable_data = [
            sylls_from_bout(wav_file, _mel_basis, hparams, visualize=visualize)
            for wav_file in tqdm(wav_list)
        ]
        
    # unpack/flatten data grabbed from loop
    for dtype, darray in zip(key_list, list(zip(*syllable_data))):
            [indv_data[dtype].extend(element) for element in darray] 
            indv_data[dtype] = np.array(indv_data[dtype])
    
    if save_dataset:
        save_syllable_dataset(individual_dataset_loc, indv_data)

Bird4 320



NameError: name 'Parallel' is not defined

In [ ]:
bird_data_packed = parallel(
                delayed(w2s.process_bout)(wav_file,_mel_basis,hparams=hparams,submode=True, visualize = visualize) 
                     for wav_file in tqdm(wav_list))

In [ ]:
import hdbscan 
from skimage.transform import resize
import umap
import time 
import seaborn as sns

def plot_with_labels(data, labels, title = '', ax = None, figsize = (9,9)):
    palette = sns.color_palette('husl', len(np.unique(labels)))
    labs_to_numbers_dict = {l:i for i,l in enumerate(np.unique(labels))}
    np.random.shuffle(palette)
    colors = [palette[labs_to_numbers_dict[x]] if x >= 0 else (0.75, 0.75, 0.75) for x in np.array(labels)]

    if not ax: fig, ax= plt.subplots(nrows=1,ncols=1,figsize=figsize)
    ax.scatter(data.T[0], data.T[1],
               color=colors, alpha = 1, linewidth= 0, s=5)
    ax.axis('off')
    ax.set_title(title)
    if not ax: plt.show()
        
def cluster_data(data, algorithm, args, kwds, verbose = True):
    """ Cluster data using arbitrary clustering algoritm in sklearn
    """
    # Function modified from HDBSCAN python package website
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    end_time = time.time()
    if verbose: print('Clustering took {:.2f} s'.format(end_time - start_time))
    return labels

In [ ]:
if parallel:
    with Parallel() as parallel:
        results = delayed(myfunction(myarguments) for myitems in mylist)
else:
    results = [myfunction(myarguments) for myitems in mylist]

In [ ]:
def loop(myfunction, mylist, myarguments, myitems, parallel=True):
    if parallel:
        with Parallel() as parallel:
            results = delayed(myfunction(myarguments) for myitems in mylist)
    else:
        results = [myfunction(myarguments) for myitems in mylist]

In [ ]:
key_list = (
            'all_bird_wav_file', # Wav file (bout_raw) that the syllable came from
            'all_bird_syll', # spectrogram of syllable
            'all_bird_syll_start', # time that this syllable occured
            'all_bird_t_rel_to_file', # time relative to bout file that this 
            'all_bird_syll_lengths' # length of the syllable
           ) 

for bird_folder in tqdm(bird_folders):
    bird_name = bird_folder.split('/')[-1]
    print(bird_name)

    # prepare the data folder
    if not os.path.exists(''.join([data_output_location,species,'/'])):
        os.makedirs(''.join([data_output_location,species,'/'])) 
    output_filename = ''.join([data_output_location,species,'/',bird_name,'_'+str(syll_size)+'.hdf5'])
    if os.path.exists(output_filename) and skip_existing:
        print('%s already complete, skipping' % (bird_name))
        continue
    
    
    # initialize lists of bird information
    bird_data = {key : [] for key in key_list}
    wav_list = glob(bird_folder + '/wavs/*.wav')
    print(bird_name, len(wav_list))
    
    if visualize==True: wav_list = wav_list[:nex]

    # Syllabify/create dataset
    if parallel:
        with Parallel(n_jobs=n_jobs, verbose=verbosity) as parallel:
            bird_data_packed = parallel(
                delayed(w2s.process_bout)(wav_file,_mel_basis,hparams=hparams,submode=True, visualize = visualize) 
                     for wav_file in tqdm(wav_list))
    else:
        bird_data_packed = [w2s.process_bout(wav_file,_mel_basis, hparams=hparams,submode=True, visualize = visualize) for wav_file in tqdm(wav_list)]
    
    if np.sum([i[0] != [] for i in bird_data_packed]) == 0:
        print('Bird had no good bouts')
        continue
        
    for dtype, darray in zip(key_list, list(zip(*bird_data_packed))):
            [bird_data[dtype].extend(element) for element in darray] # flatten and clear darray -> bird_data[dtype]
            bird_data[dtype] = np.array(bird_data[dtype])

    # reformat bird syllables
    print('len dataset: ', len(bird_data['all_bird_syll_lengths']))
    
    # embed
    x = bird_data['all_bird_syll'].reshape((len(bird_data['all_bird_syll']), syll_size,syll_size))

    x_small = [resize(i, [16,16]) for i in tqdm(x)]
    x_small = np.array(x_small).reshape((len(x_small), np.prod(np.shape(x_small)[1:])))
    x_small = [(i*255).astype('uint8') for i in x_small]

    z = umap.UMAP(
        n_neighbors=30,
        #min_dist=0.0,
        n_components=2,
        random_state=42,
    ).fit_transform(x_small)
    #label
    labels = cluster_data(z,
          hdbscan.HDBSCAN,
          (),
          {'min_cluster_size':100,  'min_samples':1},
         verbose = True)
    # plot clusters
    plot_with_labels(np.array(list(z)), labels, figsize=(20,20))
    plt.show()

    if save:
        w2s.save_dataset(output_filename, 
                     bird_data['all_bird_syll'], 
                     bird_data['all_bird_syll_start'].astype('object'),
                     bird_data['all_bird_syll_lengths'], 
                     bird_data['all_bird_wav_file'].astype('object'),
                     bird_data['all_bird_t_rel_to_file'],
                     bird_name
                    )

